In [5]:
from web3 import Web3
import json
import time
import heapq

# Configuration
arbitrum_sepolia_rpc_url = "https://sepolia-rollup.arbitrum.io/rpc"
market_contract_address = "0xeAAb3abF727D773dAFb519C42CfEAc44Cb028dDF"

# Load ABI from file
with open('../ComputationMarketABI.json', 'r') as abi_file:
    market_contract_abi = json.load(abi_file)

# Connect to Arbitrum Sepolia
web3 = Web3(Web3.HTTPProvider(arbitrum_sepolia_rpc_url))
assert web3.is_connected(), "Failed to connect to Arbitrum Sepolia"

# Create contract instance
market_contract = web3.eth.contract(address=market_contract_address, abi=market_contract_abi)

verifications = []
in_market = {}

def handle_alert_verifiers(event):
    requestId = event['args']['requestId']
    provider = event['args']['provider']
    verificationPrice = event['args']['verificationPrice']
    verificationDeadline = event['args']['verificationDeadline']
    timeAllocatedForVerification = event['args']['timeAllocatedForVerification']
    numVerifiers = event['args']['numVerifiers']
    
    print(f"AlertVerifiers event received: requestId={requestId}, provider={provider}, verificationPrice={verificationPrice}, verificationDeadline={verificationDeadline}, timeAllocatedForVerification={timeAllocatedForVerification}, numVerifiers={numVerifiers}")
    heapq.heappush(verifications, (verificationPrice, requestId, numVerifiers))
    if not requestId in in_market:
        in_market[requestId] = 0
    in_market[requestId] += numVerifiers
    print(f"Updated verifications after new verifiers alert event: {verifications}")

def handle_verification_applied(event):
    requestId = event['args']['requestId']
    verifier = event['args']['verifier']
    layerComputeIndex = event['args']['layerComputeIndex']

    print(f"VerificationApplied event received: requestId={event['args']['requestId']}, verifier={event['args']['verifier']}, layerComputeIndex={event['args']['layerComputeIndex']}")

    if requestId in in_market:
        in_market[requestId] -= 1
        if in_market[requestId] == 0:
            del in_market[requestId]
    print(f"Updated verifications after verification applied event: {verifications}")

    
def get_cheapest_verification():
    if len(verifications) > 0:
        while len(verifications) > 0 and verifications[0][1] not in in_market:
            heapq.heappop(verifications)
        if len(verifications) > 0:
            return heapq.heappop(verifications)
        return None
    else:
        return None

def listen_to_events():
    latest_block = web3.eth.block_number

    while True:
        try:
            new_block = web3.eth.block_number

            if new_block > latest_block:
                alert_verifiers_events = market_contract.events.AlertVerifiers().get_logs(fromBlock=latest_block + 1, toBlock=new_block)
                verification_applied_events = market_contract.events.VerificationApplied().get_logs(fromBlock=latest_block + 1, toBlock=new_block)

                for event in alert_verifiers_events:
                    handle_alert_verifiers(event)

                for event in verification_applied_events:
                    handle_verification_applied(event)

                latest_block = new_block

            time.sleep(1)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            time.sleep(1)

if __name__ == "__main__":
    listen_to_events()

AlertVerifiers event received: requestId=0, provider=0xC7D6129946779d29C35CCd95c984cD1fF0633678, verificationPrice=10000000000000000000, verificationDeadline=8922875208, timeAllocatedForVerification=100000, numVerifiers=5
Updated verifications after new verifiers alert event: [(10000000000000000000, 0, 5)]
VerificationApplied event received: requestId=0, verifier=0x5F373754819cCA00230eCFBE55419d76329b585A, layerComputeIndex=0
Updated verifications after verification applied event: [(10000000000000000000, 0, 5)]
VerificationApplied event received: requestId=0, verifier=0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6, layerComputeIndex=0
Updated verifications after verification applied event: [(10000000000000000000, 0, 5)]
VerificationApplied event received: requestId=0, verifier=0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561, layerComputeIndex=0
Updated verifications after verification applied event: [(10000000000000000000, 0, 5)]
VerificationApplied event received: requestId=0, verifier=0x4400B

KeyboardInterrupt: 